# Graph Construction

This notebook construct the academic graph into `torch.geometric.data` object's format given the citation information file.

#### Settings
The citation information file should have the format:
todo


#### Prepration

In [22]:
import pandas as pd
import os
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
os.getcwd()
data_dir = "./data"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

#### Raw Data processing
Skip this part if data have been processed before.

In [39]:
raw_papers = pd.read_csv("data/raw/Raw_Paper_Info.csv")
raw_papers.columns = raw_papers.columns.str.strip()

df_reIndexMapping = pd.DataFrame({"AG_Index":raw_papers.index}, index=raw_papers.Paper_ID)
reIndexMapping = dict(zip(raw_papers.Paper_ID, raw_papers.index))

df_reIndexMapping.to_csv("data/IndexMapping.csv")

In [40]:
raw_citations = pd.read_csv("data/raw/Raw_Citations.csv")
raw_citations.columns = raw_citations.columns.str.strip()

citations = pd.DataFrame({"Paper_ID": raw_citations.Paper_ID.map(reIndexMapping).copy(),
                          "Ref_Paper_ID": raw_citations.Ref_Paper_ID.map(reIndexMapping).copy()})
citations.to_csv("data/Citations.csv", index=None)

In [60]:
raw_embedding = pd.read_csv("rawdata/Raw_Paper_Embedding.csv")
raw_embedding.Paper_ID = raw_embedding.Paper_ID.map(reIndexMapping)
embedding = raw_embedding.copy()
embedding.to_csv("data/Embedding.csv", index=None)

FileNotFoundError: [Errno 2] No such file or directory: 'rawdata/Raw_Paper_Embedding.csv'

In [64]:
from CUHKSZ_AcademicGraph import CUHKSZ_AcademicGraph

dataset = CUHKSZ_AcademicGraph(root=data_dir)
dataset.process()

1


In [63]:
pd.read_csv("data/processed/Citations.csv")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 64: invalid start byte